In [14]:
import tensorflow as tf

In [15]:
from tensorflow import keras

In [16]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten , Input
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model as M

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import warnings 
warnings.filterwarnings('ignore') 

In [18]:
def moment_complexe(image,p,q):
    n,m = image.shape
    s=0
    for x in range(n):
        for y in range(m):
            s += (x + 1j * y)**p * (x - 1j * y)**q * image[x,y]
    return s

In [19]:
from numpy import exp
p_0 = 2
def invariant_rotation(image,p,q):
    return (moment_complexe(image,p_0 -1 ,p_0) ** (p-q)) * moment_complexe(image,p,q)

In [20]:
import cmath
def invariant_complet(image,p,q):
    theta = cmath.phase(invariant_rotation(image,1,0))
    gamma = (invariant_rotation(image,0,0)) ** (1/2)
    return (gamma ** -(p+q+2) )*(exp(-(p-q)*theta*1j))*moment_complexe(image,p,q)

In [21]:
import numpy as np
import cmath

In [22]:
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10,fashion_mnist
##(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [23]:
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Define the selected classes
selected_classes = [0, 1, 2, 3, 4]

# Create masks to filter the data for the selected classes
train_mask = np.isin(y_train, selected_classes).flatten()
test_mask = np.isin(y_test, selected_classes).flatten()

# Filter the data based on the masks
x_train_subset = x_train[train_mask]
y_train_subset = y_train[train_mask]
x_test_subset = x_test[test_mask]
y_test_subset = y_test[test_mask]

# Preprocess the data
x_train_subset = x_train_subset.astype('float32') / 255.0
x_test_subset = x_test_subset.astype('float32') / 255.0

# You should one-hot encode the labels for the selected classes
y_train_subset = to_categorical(y_train_subset, num_classes=len(selected_classes))
y_test_subset = to_categorical(y_test_subset, num_classes=len(selected_classes))


In [24]:
from tensorflow.keras.applications import DenseNet121,EfficientNetB0,VGG16,MobileNetV2
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
base_model = VGG16( include_top=False,
    weights="imagenet",
    input_shape=(32,32, 3), 
    pooling=max,
)
#base_model = Resnet18(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

In [25]:
import cv2
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
import sklearn
from tensorflow.keras.applications import VGG16         
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



# Extract CNN features using a small CNN-based classifier
# Function to convert the image to grayscale
def convert_to_grayscale(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayscale_image

# Function to convert images to binary or grayscale
def convert_to_binary_or_grayscale(images, use_binary=True, threshold_value=128):
    processed_images = []
    for img in images:

        gray_img = convert_to_grayscale(img)

        binary_img = convert_to_binary(gray_img, threshold_value) if use_binary else gray_img
        processed_images.append(binary_img)
    return np.array(processed_images)
def extract_cnn_features(images):
    # Extract features using the small CNN-based model
    cnn_features = base_model.predict(images)
    return cnn_features

# Compute moment invariants for each image
def compute_moment_invariants(images):
    moment_invariants_list = []
    for img in images:

        grayscale_image = convert_to_grayscale(img)
        # Compute moment invariants using OpenCV or any other library
        moments_inv = invariant_complet(grayscale_image,2,2)#moments = cv2.moments(grayscale_image)
        
        moment_invariants_complexes = moments_inv.flatten()#moment_invariants = cv2.HuMoments(moments).flatten()
        moment_invariants_list.append(moment_invariants_complexes)
        #moments = cv2.moments(grayscale_image)
        #moment_invariants_hu = cv2.HuMoments(moments).flatten()
        #moment_invariants_list.append(moment_invariants_hu)
    return np.array(moment_invariants_list)

# Function to convert the image to grayscale
def convert_to_grayscale(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return grayscale_image

# Preprocess the CIFAR-10 dataset and obtain 'images' and 'labels'
images = x_train_subset
labels = np.argmax(y_train_subset, axis=1)
#labels = np.argmax(y_train, axis=0)  # Convert one-hot encoded labels to integer labels

# Extract CNN features and moment invariants
cnn_features = extract_cnn_features(images)
moment_invariants = compute_moment_invariants(images) 
#num_moment_invariants = moment_invariants.shape[0]
num_moment_invariants = moment_invariants.shape[0]
# Reshape cnn_features to (num_moment_invariants, -1), where -1 will infer the remaining dimensions
cnn_features_flattened = cnn_features.reshape(num_moment_invariants, -1)

# Combine features
combined_features = np.concatenate([cnn_features_flattened,moment_invariants], axis=1)  

In [26]:
import numpy as np
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping


# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2)

# Create a new model for classification
classifier = Sequential()
classifier.add(Dense(units=1024, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dense(units=1024, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dense(units=256, activation='relu'))
classifier.add(Dense(5, activation='softmax'))

# Compile the model
optimizer = Adam(lr=0.001)
classifier.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = classifier.fit(x_train,
                         y_train,
                         batch_size=64,
                         epochs=30,
                         validation_data=(x_test, y_test)
                         )

# Evaluate the classifier
accuracy = classifier.evaluate(x_test, y_test, verbose=1)[1]
#print("Test Accuracy:", accuracy)
print("Best accuracy:", np.max(history.history["val_accuracy"]))



Epoch 1/30
313/313 [==============================] - 6s 18ms/step - loss: 1.0579 - accuracy: 0.6391 - val_loss: 0.8745 - val_accuracy: 0.6750
Epoch 2/30
313/313 [==============================] - 4s 13ms/step - loss: 0.7573 - accuracy: 0.7192 - val_loss: 0.8189 - val_accuracy: 0.7066
Epoch 3/30
313/313 [==============================] - 4s 13ms/step - loss: 0.6595 - accuracy: 0.7473 - val_loss: 0.7777 - val_accuracy: 0.7056
Epoch 4/30
313/313 [==============================] - 7s 21ms/step - loss: 0.6135 - accuracy: 0.7676 - val_loss: 0.8695 - val_accuracy: 0.7000
Epoch 5/30
313/313 [==============================] - 5s 17ms/step - loss: 0.5630 - accuracy: 0.7851 - val_loss: 0.8183 - val_accuracy: 0.7210
Epoch 6/30
313/313 [==============================] - 4s 13ms/step - loss: 0.5297 - accuracy: 0.7952 - val_loss: 0.8417 - val_accuracy: 0.7088
Epoch 7/30
313/313 [==============================] - 4s 13ms/step - loss: 0.4852 - accuracy: 0.8165 - val_loss: 0.8555 - val_accuracy: 0.7086